### Multioutput linear regression with Darts dapter - not working

In [ ]:
!pip install -U darts
!pip install -U u8darts>=0.29

In [ ]:
from darts.models import LinearRegressionModel
from sktime.forecasting.base.adapters._darts import _DartsRegressionAdapter

base_model = LinearRegressionModel(lags=list(range(-60,-1)))  # Example darts model with lags
darts_model = _DartsRegressionAdapter()

evaluate_linear(model = darts_model, y_train=df_pasta_train[['value_B1']], y_test=df_pasta_test[['value_B1']], lin_reg=True)

### Linear regression but for a single value - working but not wanted

In [ ]:
# Code source: Jaques Grobler
# License: BSD 3 clause

import matplotlib.pyplot as plt
import numpy as np

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

X = df_pasta_train['value_B1']
y = X.shift(df_pasta_test.shape[0]).dropna()

X = X.to_numpy().reshape(-1, 1)[:-df_pasta_test.shape[0]]
y = y.to_numpy()

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X, y)

y_test = df_pasta_test['value_B1'].to_numpy().reshape(-1, 1)

# Make predictions using the testing set
X_test = X[:365]
y_pred = regr.predict(X_test)



mase = mean_absolute_scaled_error(y_true=df_pasta_test['value_B1'], y_pred=y_pred, y_train=df_pasta_train['value_B1'])
print(f"MASE: {mase:.2f}")

### Multioutput linear regression

In [ ]:
import pandas as pd
import numpy as np
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

data = df_pasta_train

# Step 1: Create Features and Multioutput Targets
def create_multioutput_features(df, df_column, input_window=30, output_horizon=365):
    df = df.copy()
    features = []
    targets = []
    for i in range(len(df) - input_window - output_horizon):
        features.append(df[df_column].iloc[i : i + input_window].values)
        targets.append(df[df_column].iloc[i + input_window : i + input_window + output_horizon].values)
    feature_df = pd.DataFrame(features)
    target_df = pd.DataFrame(targets)
    return feature_df, target_df

input_window = 300  # Past 30 days as features
output_horizon = 365  # Forecast next 365 days
df_column = "value_B4"
X, y = create_multioutput_features(data, df_column, input_window, output_horizon)

base_model = LinearRegression()
multioutput_model = MultiOutputRegressor(base_model)
multioutput_model.fit(X, y)

# Step 4: Forecast the Next Year
last_input = data[df_column].iloc[-input_window:].values.reshape(1, -1)  # Last 30 days as input
y_pred = multioutput_model.predict(last_input).flatten()


# Step 5: Evaluate the Model
mase = mean_absolute_scaled_error(df_pasta_test[df_column], y_pred, y_train=df_pasta_train[df_column])
print(f"MASE: {mase:.2f}")

# Step 6: Plot Results
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 6))
plt.plot(data.index, data[df_column], label="True Values")
forecast_dates = pd.date_range(start="2018-01-01", periods=output_horizon, freq="D")
plt.plot(forecast_dates, y_pred, label="Forecast", color="orange")
plt.xlabel("Date")
plt.ylabel("Value")
plt.legend()
plt.grid()
plt.title("Multioutput Regression Forecast for Next Year")
plt.show()


### Linear regression from darts without CV

In [ ]:
import numpy as np
from darts.models import LinearRegressionModel
from sktime.forecasting.base import ForecastingHorizon

# Step 2: Initialize the Darts Model
# Properly initialize LinearRegressionModel with valid lags
base_model = LinearRegressionModel(lags=1, output_chunk_length=366)  # Lags for autoregression

adapter = DartsForecaster(model=base_model)
fh = ForecastingHorizon(np.arange(1, df_pasta_test.shape[0]+1))

# Step 5: Fit the Model
# Ensure `y_train` is a pandas Series and matches sktime expectations
adapter.fit(df_pasta_train, fh=fh)

# Step 6: Make Predictions
y_pred = adapter.predict(fh=fh)

mase = mean_absolute_scaled_error(y_true=df_pasta_test, y_pred=y_pred, y_train=df_pasta_train)
print(f"MASE: {mase:.2f}")